In [249]:
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import Sequential,layers
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
# import keras
from keras import backend as K
from src import data
from src import error_function

import IPython.display as ipd
import os 
import glob 
import pdb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [219]:
!ls ../data/merged_outputtrain/

0  1  10  2  3	4  5  6  7  8  9


In [220]:
!ls ../data/spectrograms/9

9_1008.png  9_1268.png	9_1518.png  9_1778.png	9_288.png  9_538.png  9_78.png
9_1018.png  9_1278.png	9_1528.png  9_1788.png	9_28.png   9_548.png  9_798.png
9_1028.png  9_1288.png	9_1538.png  9_178.png	9_298.png  9_558.png  9_808.png
9_1038.png  9_128.png	9_1548.png  9_1798.png	9_308.png  9_568.png  9_818.png
9_1048.png  9_1298.png	9_1558.png  9_1808.png	9_318.png  9_578.png  9_828.png
9_1058.png  9_1308.png	9_1568.png  9_1818.png	9_328.png  9_588.png  9_838.png
9_1068.png  9_1318.png	9_1578.png  9_1828.png	9_338.png  9_58.png   9_848.png
9_1078.png  9_1328.png	9_1588.png  9_1838.png	9_348.png  9_598.png  9_858.png
9_1088.png  9_1338.png	9_158.png   9_1848.png	9_358.png  9_608.png  9_868.png
9_108.png   9_1348.png	9_1598.png  9_1858.png	9_368.png  9_618.png  9_878.png
9_1098.png  9_1358.png	9_1608.png  9_1868.png	9_378.png  9_628.png  9_888.png
9_1108.png  9_1368.png	9_1618.png  9_1878.png	9_388.png  9_638.png  9_88.png
9_1118.png  9_1378.png	9_1628.png  9_1888.png	9_38.png   9_648.png  

In [221]:
def obtain_label(filename):
    return filename.split("/")[-1].split("_")[0]

In [284]:
def parse_function(filename, label):
#     audio_sample, sample_rate = tf.audio.decode_wav(filename)    
    audio_binary = tf.read_file(filename)
    audio_decoded = tf.contrib.ffmpeg.decode_audio(
        audio_binary,
        file_format="wav",
        samples_per_second=16000,
        channel_count=1
    )
    return tf.cast(audio_decoded, tf.float32),label
    

In [285]:
def data_generator():
    filenames = glob.glob('../data/merged_outputtrain/*/*')
    labels = list(map(obtain_label, filenames))
    for filename,label in zip(filenames,labels):
        yield filename,label

In [354]:
def select_random_excerpt(file,label): #length can be changed..
    excerpt_duration=1
    excerpt_duration_frames = excerpt_duration * 16000
    limit = 80000 - excerpt_duration_frames
    start = np.random.randint(0,limit)
    end = start + excerpt_duration_frames
#     pdb.set_trace()
    return tf.slice(tf.squeeze(file), [start], [excerpt_duration_frames]),label

In [ ]:
def 

In [355]:
filenames = glob.glob('../data/merged_outputtrain/*/*')
labels = list(map(obtain_label, filenames))
dataset = tf.data.Dataset.from_tensor_slices( (filenames,labels) )
dataset = dataset.map(parse_function, num_parallel_calls=4)
dataset = dataset.map(select_random_excerpt, num_parallel_calls=4)
dataset = dataset.batch(32)
dataset = dataset.prefetch(1)

In [356]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()


In [357]:
with tf.Session() as sess:
    for i in range(10):
        z = next_element
        t = sess.run(z)

In [360]:
ipd.Audio(t[0][12].T, rate=16000)

In [361]:
len(t[0][12])

16000

In [325]:
for i in range(100000):
    start = np.random.randint(0,(80000-16000))
    
    end = start + 16000
    assert end <= 80000

<function tensorflow.python.ops.array_ops.slice(input_, begin, size, name=None)>